In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('dailymilk_production.csv', index_col='Date',parse_dates=True)
df.index.freq = 'D'

In [5]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit


In [6]:
# Define number of splits and window size
n_splits = 5
window_size = 10

In [7]:
# Define the cross-validation iterator
tscv = TimeSeriesSplit(n_splits=n_splits)

In [10]:
train = df.iloc[:156]
test = df.iloc[156:]

In [8]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [11]:
scaler.fit(train)
scaled_train = scaler.transform(train)
scaled_test = scaler.transform(test)

In [12]:
from keras.preprocessing.sequence import TimeseriesGenerator

In [13]:
#define generator
n_input = 10
n_features = 1
generator = TimeseriesGenerator(scaled_train,scaled_train,length =n_input,batch_size=1)

In [14]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [16]:
# Build model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [17]:
# Train model
model.fit(generator, epochs=120)

Epoch 1/120
146/146 [==============================] - 1s 3ms/step - loss: 0.1089
Epoch 2/120
146/146 [==============================] - 0s 3ms/step - loss: 0.0995
Epoch 3/120
146/146 [==============================] - 0s 3ms/step - loss: 0.0916
Epoch 4/120
146/146 [==============================] - 0s 3ms/step - loss: 0.0895A: 0s - loss:
Epoch 5/120
146/146 [==============================] - 0s 3ms/step - loss: 0.0902
Epoch 6/120
146/146 [==============================] - 0s 3ms/step - loss: 0.0934
Epoch 7/120
146/146 [==============================] - 1s 4ms/step - loss: 0.0920
Epoch 8/120
146/146 [==============================] - 1s 4ms/step - loss: 0.0894A: 0s - loss: 0.
Epoch 9/120
146/146 [==============================] - 0s 3ms/step - loss: 0.0909
Epoch 10/120
146/146 [==============================] - 0s 3ms/step - loss: 0.0898
Epoch 11/120
146/146 [==============================] - 0s 3ms/step - loss: 0.0898
Epoch 12/120
146/146 [==============================] - 0s 3ms/step

In [18]:
# Evaluate model on test set
test_generator = TimeseriesGenerator(scaled_test, scaled_test, length=n_input, batch_size=1)
test_mse = model.evaluate(test_generator)
print(f'Test set MSE: {test_mse:.3f}')

18/18 [==============================] - 0s 2ms/step - loss: 0.0454
Test set MSE: 0.045


In [19]:
# Cross-validation
n_splits = 5
tscv = TimeSeriesSplit(n_splits=n_splits)
cv_mses = []
for train_index, test_index in tscv.split(df):
    # Split the data into training and test sets
    train = df.iloc[train_index]
    test = df.iloc[test_index]

    # Scale the data
    scaler.fit(train)
    scaled_train = scaler.transform(train)
    scaled_test = scaler.transform(test)

    # Generate sequences
    train_generator = TimeseriesGenerator(scaled_train, scaled_train,
                                           length=n_input, batch_size=1)
    test_generator = TimeseriesGenerator(scaled_test, scaled_test,
                                          length=n_input, batch_size=1)

    # Build the model
    model = Sequential()
    model.add(LSTM(100, activation='relu', input_shape=(n_input, 1)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    # Train the model
    model.fit(train_generator, epochs=120, verbose=0)

    # Evaluate the model on the test set
    mse = model.evaluate(test_generator, verbose=0)
    cv_mses.append(mse)


In [ ]:
from keras.regularizers import L2
from keras.layers import Dropout

model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features), 
               kernel_regularizer=L2(0.001), recurrent_regularizer=L2(0.001)))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [20]:
# Print cross-validation results
mean_cv_mse = np.mean(cv_mses)
std_cv_mse = np.std(cv_mses)
print(f'Cross-validation results:')
print(f'Mean MSE: {mean_cv_mse:.3f}')
print(f'STD MSE: {std_cv_mse:.3f}')

Cross-validation results:
Mean MSE: 0.140
STD MSE: 0.050
